In [1]:
import folium
import pandas as pd
import numpy as np
import re


# Interactive map of individual parking citations
We draw a map of individaul 2019 parking citations in San Francisco based on the time of day.

### Process data
We subset a random sample of the data, and process it to include longitude and latitude coordinates as seperate columns using regular expressions. 

In [2]:
# Import 2019 data with geometries
df = pd.read_csv('csv/parking_citations_w_geom_2019.csv')
# Take random sample to map
df = df.sample(frac=0.001, random_state=0)


In [3]:
# Dataframe dimensions
length = df.shape[0]
width = df.shape[1]
df.shape

(1069, 10)

In [4]:
# Get latitude and longitude coordinates in seperate columns using regex
for i in range(length):
    coordinates = re.findall('[-0-9.]+', df.loc[df.index[i], 'geom'])
    df.loc[df.index[i], 'lat'] = coordinates[1]
    df.loc[df.index[i], 'lon'] = coordinates[0]

In [5]:
# Drop geom column
df = df.drop(columns=['geom'])

In [6]:
# Convert to pandas datetime
df.loc[:, 'Citation_Issued_DateTime'] = pd.to_datetime(df['Citation_Issued_DateTime'])

In [7]:
# Check for NA's: there are no NA's for desired columns
df.isnull().sum(axis = 0)

Citation_Number              0
Citation_Issued_DateTime     0
Violation                    0
Violation_Description        0
Citation_Location            1
Vehicle_Plate_State         44
Vehicle_Plate               46
Fine_Amount                  0
Date_Added                   0
lat                          0
lon                          0
dtype: int64

In [8]:
# Check work
df.head()

,Citation_Number,Citation_Issued_DateTime,Violation,Violation_Description,Citation_Location,Vehicle_Plate_State,Vehicle_Plate,Fine_Amount,Date_Added,lat,lon
1019861,PD34613596,2019-08-13 14:05:00,V5202,NOPL/PRDSP,IRONWOOD AT CATALINA,CA,4XNF490,121.0,2020-04-30T00:00:00,37.73629004500003,-122.38011988899996
899663,925928813,2019-12-04 10:23:00,TRC7.2.70,OBSTRCT TF,233 SERRANO DR,CA,8BDZ371,110.0,2020-07-27T00:00:00,37.71971298700004,-122.47937603999998
973061,909704062,2019-05-01 14:47:00,TRC7.2.23B,MTR OUT DT,317 09TH ST,CA,7MQS630,76.0,2020-04-30T00:00:00,38.24842055800008,-122.036426329
208804,1005658183,2019-03-07 13:56:00,TRC7.2.38,PK STANDS,331 TOWNSEND,CA,7NDE878,110.0,2022-04-11T00:00:00,37.776536270000065,-122.39560939499995
688599,911650331,2019-07-09 12:18:00,TRC7.2.22,STR CLEAN,962 PAGE ST,CA,22T8130,79.0,2020-07-27T00:00:00,37.77256499400005,-122.43665800499997


### Create map
Subsets individual parking citations by time of day

In [11]:
# Create folium map
map = folium.Map(location = [37.7749, -122.4194], zoom_start = 12)

In [ ]:
def popup_html(i, data):
    """Define function to parse HTML for folium popup. i is row of citation data (df)"""
    
    df = data
    fine_amount = df['Fine_Amount'].iloc[i]
    citation_location = df['Citation_Location'].iloc[i]
    violation_desc = df['Violation_Description'].iloc[i]
    datetime = df['Citation_Issued_DateTime'].iloc[i]


    html = f"""<!DOCTYPE html>
    <html>
    <table style="height: 120px; width: 360px;">
    <tbody>
    <tr>
    <td>Citation Location</td>
    <td>{citation_location}</td>
    </tr>
    <tr>
    <td>Citation Amount</td>
    <td>{fine_amount}</td>
    </tr>
    <tr>
    <td>Violation Description</td>
    <td>{violation_desc}</td>
    </tr>
    <tr>
    <td>Datetime</td>
    <td>{datetime}</td>
    </tr>
    </tbody>
    </table>
    </html>
    """
    return html

In [10]:
for i in range(len(df.index)):
    lat = df.iloc[i, -2]
    lon = df.iloc[i, -1]
    coordinates = [float(lat), float(lon)]

    # create html table for current data point; add as popup
    popup_html = popup_html(i, df)
    popup = folium.Popup(popup_html, parse_html = True)

    # add parking citation as folium Circle
    circle = folium.Circle(coordinates, color = "red", radius = 10, popup = popup)



    

In [10]:
fig = folium.Figure(width = 900, height = 400)
fig.add_child(map)